## The imports

Basically we use the keras library for the problem. In addition to this we use scikit-image for image processing.

In [1]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import tensorflow as tf

Using TensorFlow backend.


## Basic variables

In this section we set the basic variables for the data, like dataset size, valid ant test split.
(We decided to create a variable for the data size because we didn't run the training on the whole dataset each time) 

In [3]:
dataset_size = 70000

#Since we renamed the images to numbers from 1 to ~70k we use a range to index the images.
ids = list(range(0, 70000))
#Shuffling the ids into a random order to shuffle the images.
random.shuffle(ids)

all_id = ids[:dataset_size]

#Test/Valid splits
valid_split = 0.1
test_split = 0.1

v_index = int(len(all_id)*(1-valid_split-test_split))
t_index = int(len(all_id)*(1-test_split))

#Setting the splits
train_ids = all_id[:v_index]
valid_ids = all_id[v_index:t_index]
test_ids = all_id[t_index:]

print("Length of train set: " + str(len(train_ids)))
print("Length of validation set: " + str(len(valid_ids)))
print("Length of test set: " + str(len(test_ids)))

partition = {'train': train_ids, 'validation': valid_ids, 'test': test_ids}

Length of train set: 56000
Length of validation set: 7000
Length of test set: 7000


## The DataGenerator class

You can read the details of it in the documentation.

In [4]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=10, dim=(256,256,1), shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *(256,256,1)), dtype=float)
        Y = np.empty((self.batch_size, *(256,256,2)), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            img = img_to_array(load_img('C:/Users/lakas/Documents/deeplearning/imcol-master/dataset/' + str(ID) + '.jpg'))
            img = 1.0/255*img
            img = rgb2lab(img)
            
            gray = img[:,:,0] 
            ab = img[:,:,1:] / 128
            
            # Store sample
            X[i,] = gray.reshape((256,256,1))

            # Store class
            Y[i,] = ab

        return X, Y

## Early stopping

we used an early stopping with a patienc of 15. In addition to preventing the network from overfitting it also made our program more robust since in case of an error during the training we didn't loose the weights.

In [5]:
patience=15
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

## The swish activation

Here you can see the implementation of the swish function. More details about the function in the documentary.
The Swish class is a specialization of the Activation class. You can reach it in the keras methods just like you can e.g 'relu'. (it's name is: 'swish')

In [6]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

class Swish(Activation):
    
    def __init__(self, activation, **kwargs):
        super(Swish, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'

def swish(x):
    A = (K.sigmoid(x) * x)
    return A

get_custom_objects().update({'swish': Swish(swish)})

## Building the network

Here we build the network with the given activation in all layers.
In our opinion this is the best resulting network, but since it is subjective which result images you think are the best you can find the differnet networks' results in the documentary.

In [7]:
act = 'swish'

#Encoder
encoder_input = Input(shape=(256, 256, 1,), name='encoder_input')
encoder_output = Conv2D(64, (3,3), activation=act, padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation=act, padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation=act, padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation=act, padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation=act, padding='same', strides=2)(encoder_output)

#Bottle-neck
encoder_output = Conv2D(512, (3,3), activation=act, padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation=act, padding='same')(encoder_output)


#Decoder
encoder_output = Conv2D(256, (3,3), activation=act, padding='same')(encoder_output)
decoder_output = Conv2D(128, (3,3), activation=act, padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation=act, padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation=act, padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation=act, padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=encoder_input, outputs=decoder_output)

## Setting the generators and starting the training

We used a batch-size of 50, adam optimizer, and mean_absolute_error.

In [8]:
training_generator = DataGenerator(partition['train'], batch_size=50)
valid_generator = DataGenerator(partition['validation'], batch_size=50)
# Train model      
model.compile(optimizer='adam', loss='mean_absolute_error')
model.fit_generator(training_generator, validation_data=valid_generator,
                    epochs=40, steps_per_epoch=len(train_ids)//50, validation_steps=len(valid_ids)//50)#,
                   #callbacks=[checkpointer, early_stopping])

Epoch 1/40
1120/1120 [==============================] - 1305s 1s/step - loss: 0.1024 - val_loss: 0.0960
Epoch 2/40
1120/1120 [==============================] - 1261s 1s/step - loss: 0.0960 - val_loss: 0.0955
Epoch 3/40
1120/1120 [==============================] - 1252s 1s/step - loss: 0.0957 - val_loss: 0.0959
Epoch 4/40
1120/1120 [==============================] - 1248s 1s/step - loss: 0.0949 - val_loss: 0.0942
Epoch 5/40
1120/1120 [==============================] - 1247s 1s/step - loss: 0.0943 - val_loss: 0.0940
Epoch 6/40
1120/1120 [==============================] - 1249s 1s/step - loss: 0.0937 - val_loss: 0.0934
Epoch 7/40
1120/1120 [==============================] - 1248s 1s/step - loss: 0.0934 - val_loss: 0.0926
Epoch 8/40
1120/1120 [==============================] - 1254s 1s/step - loss: 0.0929 - val_loss: 0.0922
Epoch 9/40
1120/1120 [==============================] - 1246s 1s/step - loss: 0.0924 - val_loss: 0.0916
Epoch 10/40
1120/1120 [==============================] - 1245s 1

## Saving the model

We both save the model into a .json file and the weights into an .hdf5 file.

In [9]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("weights.hdf5")

## Loading the model

We load the model from the json file in order to be able to run the testing without running the whole program again.

In [23]:
from keras.models import model_from_json
import json

with open('model.json') as json_data:
    d = json_data.read()


json_string = d
model = model_from_json(json_string)

## Testing the model

It gets the test images from the Test/ directory and place the results into the result/ directory.

In [24]:
# Change to '/data/images/Test/' to use all the 500 images

model.load_weights('model.h5')

color_me = []
for filename in os.listdir('Test/'):
	color_me.append(img_to_array(load_img('Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0] #Only using the gray channel as the input
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
#output = model.predict(color_me / 100) #Norm
output = model.predict(color_me) #No Norm
output = output * 128

# Output colorizations
for i in range(len(output)):
	cur = np.zeros((256, 256, 3))
	cur[:,:,0] = color_me[i][:,:,0]
	cur[:,:,1:] = output[i]
	imsave("result/img_"+str(i)+".png", lab2rgb(cur))

## Model summary

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 512)       1180160   
__________

## Model memory usage

We found a little code that prints out the memory usage of the model.

In [13]:
def get_model_memory_usage(batch_size, model):
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

get_model_memory_usage(100, model)

5.772